In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 

tablename = "dimdivisions"
runcycleid = etl_helpers.start_run_cycle(f"{tablename}")
os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    spark.sql(f"DROP TABLE IF EXISTS {tablename};")

    query = f"""
        CREATE TABLE {tablename}
        USING DELTA AS
            SELECT
                a.divisionid,
                a.name AS division,
                NULL AS description,
                CASE
                    WHEN a.isactive = TRUE THEN 'Y'
                ELSE
                    'N'
                END AS cactive,
                b.bcgovcode AS programarea
                FROM foi_mod.programareadivisions a
                INNER JOIN foi_mod.programareas b
                ON a.programareaid = b.programareaid
                order by a.divisionid;  
        """

    # print(query)

    df = spark.sql(query)
    df.show()

    etl_helpers.end_run_cycle(runcycleid, 't', f"{tablename}")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', f"{tablename}", "Credentials not available")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', f"{tablename}")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', f"{tablename}", f"An error occurred: {e}")